<a href="https://colab.research.google.com/github/nguyenfan20/AI-Agents-using-LangChain/blob/main/Vietnamese_Legal_Traffic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tổng quan dự án 🤖

Notebook này trình bày cách tạo một chatbot bằng hệ thống RAG dựa vào thư viện LangChain và LLMs

In [1]:
!pip uninstall -y fsspec datasets gcsfs
!pip install fsspec==2024.12.0 datasets==3.5.0 gcsfs==2024.12.0
!pip install -q torch transformers accelerate bitsandbytes \
  langchain sentence-transformers faiss-cpu openpyxl pacmap datasets \
  langchain-community ragatouille tqdm pymupdf python-docx pandas

Found existing installation: fsspec 2024.12.0
Uninstalling fsspec-2024.12.0:
  Successfully uninstalled fsspec-2024.12.0
Found existing installation: datasets 3.5.0
Uninstalling datasets-3.5.0:
  Successfully uninstalled datasets-3.5.0
Found existing installation: gcsfs 2024.12.0
Uninstalling gcsfs-2024.12.0:
  Successfully uninstalled gcsfs-2024.12.0
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached gcsfs-2024.12.0-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
Using cached gcsfs-2024.12.0-py2.py3-none-any.whl (35 kB)


In [2]:
import os
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple

FILE_01 = '/content/luatgt2.pdf'


VECTOR_DATABASE_PATH = '/content/vectordatabase'
os.makedirs('/content/vectordatabase', exist_ok=True)

# Chunking 🔪

In [3]:
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm
from langchain.document_loaders import PyMuPDFLoader
from docx import Document as DocxDocument
import pandas as pd
import os

def load_pdf_file(file_path):
    """Loads a PDF file and returns its entire content using PyMuPDFLoader."""
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    full_content = ""
    for doc in documents:
        full_content += doc.page_content + "\n"  # Add a newline to separate pages
    return full_content  # Return the entire content

def load_txt_file(file_path):
    """Loads a TXT file and returns its content as a string."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def load_docx_file(file_path):
    """Loads a DOCX file and returns its content as a string."""
    doc = DocxDocument(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

def load_csv_file(file_path):
    """Loads a CSV file and concatenates all rows as a single string."""
    df = pd.read_csv(file_path)
    return df.to_string(index=False)  # Converts the DataFrame to a string (without row indices)

def load_file(file_path):
    """Determines the file type and loads the file content."""
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.txt':
        return load_txt_file(file_path)
    elif ext in ['.doc', '.docx']:
        return load_docx_file(file_path)
    elif ext == '.pdf':
        return load_pdf_file(file_path)
    elif ext == '.csv':
        return load_csv_file(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [4]:
# List of document file paths (PDF, DOCX, TXT, CSV)
file_paths = [FILE_01] # Example file paths. It can be like: file_paths = [FILE_01, FILE_02,..]

RAW_KNOWLEDGE_BASE = []
for file_path in tqdm(file_paths):
    try:
        content = load_file(file_path)
        RAW_KNOWLEDGE_BASE.append(
            LangchainDocument(page_content=content, metadata={"source": file_path})
        )
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [5]:
for doc in RAW_KNOWLEDGE_BASE:
    print(f"Source: {doc.metadata['source']}")
    print(f"Content snippet: {doc.page_content[:500]}...\n")

Source: /content/luatgt2.pdf
Content snippet: CHÍNH PHỦ
--------
CỘNG HÒA XÃ HỘI CHỦNGHĨA VIỆT NAM
Độc lập - Tựdo - Hạnh phúc
---------------
Số: 168/2024/NĐ-CP
Hà Nội, ngày 26 tháng 12 năm 2024
NGHỊĐỊNH
QUY ĐỊNH XỬPHẠT VI PHẠM HÀNH CHÍNH VỀTRẬT TỰ, AN TOÀN GIAO
THÔNG TRONG LĨNH VỰC GIAO THÔNG ĐƯỜNG BỘ; TRỪĐIỂM, PHỤC HỒI
ĐIỂM GIẤY PHÉP LÁI XE
Căn cứLuật Tổchức Chính phủngày 19 tháng 6 năm 2015; Luật sửa đổi, bổsung một
sốđiều của Luật Tổchức Chính phủvà Luật Tổchức chính quyền địa phương ngày 22
tháng 11 năm 2019;
Căn cứLuật Xửlý vi phạm hà...



In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [7]:
from sentence_transformers import SentenceTransformer

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(
    f"Model's maximum sequence length: {SentenceTransformer('minhtt/phobert-base-v2').max_seq_length}"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at minhtt/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model's maximum sequence length: 258


# Embedding document 📂

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

EMBEDDING_MODEL_NAME = "AITeamVN/Vietnamese_Embedding"


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed = split_documents(
    258,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

# Vector Database 📂

In [10]:
import os
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)


<ipython-input-10-26065c6d9089>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


# Lưu Vector Database được xử lý vào các thư mục

In [11]:
import faiss  # Make sure to import FAISS
import pickle

# Save the FAISS index to a file
faiss_file_path = os.path.join(VECTOR_DATABASE_PATH, 'faiss_index.bin')
faiss.write_index(KNOWLEDGE_VECTOR_DATABASE.index, faiss_file_path)
print(f"FAISS index saved to {faiss_file_path}")

# Save the document store to a pickle file
docstore_file_path = os.path.join(VECTOR_DATABASE_PATH, 'docstore.pkl')
with open(docstore_file_path, 'wb') as f:
    pickle.dump(KNOWLEDGE_VECTOR_DATABASE.docstore, f)
print(f"Document store saved to {docstore_file_path}")

# Save the index_to_docstore_id mapping
mapping_file_path = os.path.join(VECTOR_DATABASE_PATH, 'index_to_docstore_id.pkl')
with open(mapping_file_path, 'wb') as f:
    pickle.dump(KNOWLEDGE_VECTOR_DATABASE.index_to_docstore_id, f)
print(f"Index to document store ID mapping saved to {mapping_file_path}")

# Load the FAISS index from the file
loaded_index = faiss.read_index(faiss_file_path)

# Load the document store from the pickle file
with open(docstore_file_path, 'rb') as f:
    loaded_docstore = pickle.load(f)

# Load the index_to_docstore_id mapping
with open(mapping_file_path, 'rb') as f:
    index_to_docstore_id = pickle.load(f)

# Create the FAISS vector store using the loaded index and document store
loaded_vector_database = FAISS(
    index=loaded_index,
    docstore=loaded_docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query
)
print("Vector database loaded successfully.")

FAISS index saved to /content/vectordatabase/faiss_index.bin
Document store saved to /content/vectordatabase/docstore.pkl
Index to document store ID mapping saved to /content/vectordatabase/index_to_docstore_id.pkl
Vector database loaded successfully.


# Chain với thư mục đã lưu

In [12]:
# Embed a user query in the same space
user_query = "Quy định về độ tuổi lái xe mô tô hai bánh?"
query_vector = embedding_model.embed_query(user_query)

In [13]:
print(f"\nStarting retrieval for {user_query=}...")
# retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)

retrieved_docs = loaded_vector_database.similarity_search(query=user_query, k=5)

print(
    "\n==================================Top document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)


Starting retrieval for user_query='Quy định về độ tuổi lái xe mô tô hai bánh?'...

==================================Top document==================================
xe).
6. Phạt tiền từ4.000.000 đồng đến 6.000.000 đồng đối với người từđủ16 tuổi đến dưới
18 tuổi điều khiển xe ô tô, xe chởngười bốn bánh có gắn động cơ, xe chởhàng bốn bánh
có gắn động cơ và các loại xe tương tựxe ô tô.
7. Phạt tiền từ6.000.000 đồng đến 8.000.000 đồng đối với người điều khiển xe mô tô hai
bánh có dung tích xi-lanh trên 125 cm3 trởlên hoặc có công suất động cơ điện trên 11
kW, xe mô tô ba bánh thực hiện một trong các hành vi vi phạm sau đây:
a) Có giấy phép lái xe nhưng không phù hợp với loại xe đang điều khiển;
b) Không có giấy phép lái xe hoặc sửdụng giấy phép lái xe đã bịtrừhết điểm, giấy phép
lái xe không do cơ quan có thẩm quyền cấp, giấy phép lái xe bịtẩy xóa, giấy phép lái xe
không còn hiệu lực;
==================================Metadata==================================
{'source': '/content/luatgt2.

# Đưa LLM để tạo sinh câu trả lời với đữ liệu được chain

In [14]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [15]:
!pip -q install langdetect

In [16]:
from langdetect import detect

prompt_in_chat_format_en = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]

# Prompt template for Vietnamese
prompt_in_chat_format_vi = [
    {
        "role": "system",
        "content": """Sử dụng thông tin trong ngữ cảnh, hãy đưa ra câu trả lời đầy đủ cho câu hỏi.
Chỉ trả lời câu hỏi được hỏi, câu trả lời cần ngắn gọn và phù hợp với câu hỏi.
Cung cấp số của tài liệu nguồn khi phù hợp.
Nếu câu trả lời không thể suy ra từ ngữ cảnh, không đưa ra câu trả lời.""",
    },
    {
        "role": "user",
        "content": """Ngữ cảnh:
{context}
---
Bây giờ đây là câu hỏi mà bạn cần trả lời.

Câu hỏi: {question}""",
    },
]

def detect_language(query):
    return detect(query)

def create_prompt(question):

    language = detect_language(question)

    if language == 'vi':
        RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
            prompt_in_chat_format_vi, tokenize=False, add_generation_prompt=True
        )
    else:
        RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
            prompt_in_chat_format_en, tokenize=False, add_generation_prompt=True
        )

    return RAG_PROMPT_TEMPLATE

#test prompt if the question is Vietnamese

test_prompt = create_prompt("Hà Nội là thủ đô của nước nào?")
print(test_prompt)

<|system|>
Sử dụng thông tin trong ngữ cảnh, hãy đưa ra câu trả lời đầy đủ cho câu hỏi.
Chỉ trả lời câu hỏi được hỏi, câu trả lời cần ngắn gọn và phù hợp với câu hỏi.
Cung cấp số của tài liệu nguồn khi phù hợp.
Nếu câu trả lời không thể suy ra từ ngữ cảnh, không đưa ra câu trả lời.</s>
<|user|>
Ngữ cảnh:
{context}
---
Bây giờ đây là câu hỏi mà bạn cần trả lời.

Câu hỏi: {question}</s>
<|assistant|>



In [17]:
retrieved_docs_text = [
    doc.page_content for doc in retrieved_docs
]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join(
    [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
)

user_query="Quy định về độ tuổi lái xe mô tô 2 bánh?"

rag_prompt = create_prompt(user_query)
final_prompt = rag_prompt.format(
    question=user_query, context=context
)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

Theo quy định, người lái xe mô tô hai bánh phải được tuổi 16 hoặc trở lên. (Điều 18, Document 2)


In [18]:
from transformers import Pipeline
from typing import Tuple, List
from langchain.schema import Document as LangchainDocument
from faiss import Index as FAISS

def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:

    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )

    # Ensure that relevant_docs is not empty or None
    if not relevant_docs:
        raise ValueError("No relevant documents retrieved.")

    # Keep only the text from the retrieved documents
    relevant_docs = [doc.page_content for doc in relevant_docs]

    # Ensure k is not larger than the number of documents
    num_docs_final = min(num_docs_final, len(relevant_docs))

    if num_docs_final < 1:
        raise ValueError("Not enough documents for processing.")

    # Limit to the final number of documents
    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    RAG_PROMPT_TEMPLATE = create_prompt(question)
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Generate the answer using the LLM
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [19]:
question = "Quy định về mức phạt với vi phạm nồng độ cồn trong máu?"

answer, relevant_docs = answer_with_rag(
    question, READER_LLM, loaded_vector_database)

=> Retrieving documents...
=> Generating answer...


In [20]:
print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)

==================================Answer==================================
Quy định về mức phạt với vi phạm nồng độcồn trong máu được điều chỉnh trong các trường hợp sau:

- Document 1: Nồng độcồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở (Điều kháu a trong các hành vi vi phạm): Phạt tiền từ 12.000.000 đồng đến 14.000.000 đồng.

- Document 2: Nồng độcồn vượt quá 80 miligam/100 mililít máu (Điều kháu a trong các hành vi vi phạm): Phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng.

- Document 3: Khi có liên quan trực tiếp đến vụ ta nạn giao thông, không dừng ngay phương tiện, không giữnguyên hiện trường, không trợgiúp người bị nạn (Điều kháu a trong các hành vi vi phạm): Phạt tiền theo quy định tại điểm c khoản 9 Điều này.

Các quy định trên có hiệu lực chính quyền nếu người điều khiển xe
==================================Source docs==================================
Document 0------------------------------------------------------------
h

In [ ]:
import os
os._exit(00)